In [4]:
import cv2
import pandas as pd
import time
import numpy as np
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import pdist, squareform

# Charger la vidéo originale
video_path = "Puente_guambra2.mp4"

# Charger les données du fichier CSV contenant les détections
data = pd.read_csv(f"{video_path}_positions.csv")

data

,frame_number,id,class,conf,x,y,w,h,r
0,1,1,small vehicle,0.802636,3148.414062,1927.137207,64.585747,30.545033,0.689725
1,1,2,small vehicle,0.795741,3054.048584,463.554932,71.559853,32.174553,2.819362
2,1,3,large vehicle,0.775441,3079.295166,1646.043091,190.205017,43.182365,0.649792
3,1,4,small vehicle,0.775234,3848.507080,297.390717,67.181137,28.472355,2.855587
4,1,5,small vehicle,0.774091,238.860870,1675.974121,68.550430,31.932125,0.058043
...,...,...,...,...,...,...,...,...,...
682993,2787,257,small vehicle,0.010771,1617.497070,981.535706,58.366802,25.187868,0.528911
682994,2787,258,small vehicle,0.010659,1522.585083,474.937256,87.213348,34.550034,2.684876
682995,2787,259,small vehicle,0.010522,3660.509521,1577.630249,36.204979,23.738024,2.863581
682996,2787,260,small vehicle,0.010248,3945.224609,213.176300,64.669098,28.852522,2.828396


In [ ]:
# Définir une liste de couleurs pour chaque classe
class_colors = {
    'small vehicle': (0, 255, 255),
    'large vehicle': (255, 255, 0),
    # Ajouter d'autres classes et couleurs au besoin
}

threshold = 10

look_back = 10


def draw_obb(img, cx, cy, w, h, angle, class_name, confidence, id, points, colors=class_colors, thickness=2):
    rect = ((cx, cy), (w, h), angle*180/np.pi)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    cv2.drawContours(img, [box], 0, colors[class_name], thickness)
    cv2.putText(img, f"{class_name} ({confidence:.2f}) id: {id}", (int(cx), int(cy) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[class_name], 2)
    cv2.polylines(img, [points], isClosed=False, color=colors[class_name], thickness=thickness)  



max_id = 1

cap = cv2.VideoCapture(video_path)

last_positions = []

# Lire la vidéo frame par frame
while cap.isOpened():
    
    ret, frame = cap.read()
    if not ret:
        break
    

    frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

    # Filtrer les détections pour le frame actuel
    detections_in_frame = data[data['frame_number'] == frame_number]

    # Dessiner les boîtes englobantes sur le frame actuel
    for index, row in detections_in_frame.iterrows():
        cx, cy, w, h, r = row['x'], row['y'], row['w'], row['h'], row['r']
        class_name, confidence = row['class'], row['conf']
        
        
        detection_center = np.array([cx, cy])
        
        id = None
        
        dist = []
        
        ids = []
        
        for last_position in last_positions:
            
            n_diff_frames = frame_number - last_position['frame_number']
            
            if n_diff_frames <= look_back:
            
                dist.append(euclidean(detection_center, last_position['center']))
                
                ids.append(last_position['id'])
                
        if len(dist) > 0:
            min_dist = min(dist)
            id = ids[dist.index(min_dist)]
            
            if min_dist < threshold:
                id = id
            else:
                id = max_id
                max_id += 1        
        else:
            id = max_id
            max_id += 1
            


            
        last_positions.append({'center': detection_center, 'id': id, 'frame_number': frame_number, 'class': class_name, 'conf': confidence, 'x': cx, 'y': cy, 'w': w, 'h': h, 'r': r})

        track =  [(p.get('x'), p.get("y")) for p in last_positions if p.get('id') == id]

        points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))


In [21]:
from sklearn.cluster import DBSCAN


threshold = 10

look_back = 20

max_id = 1

cap = cv2.VideoCapture(video_path)

last_positions = []

rolling_window = [[j+i for i in range(look_back)] for j in range(data['frame_number'].min(), data['frame_number'].max() + 1)]

# Lire la vidéo frame par frame
for frame_number in rolling_window:

    # Filtrer les détections pour le frame actuel
    frames = data[data['frame_number'].isin(frame_number)]
    
    min_cluster_size = 2
    
    max_cluster_size = look_back
    
    min_samples = 2
    
        # Apply DBSCAN
    db = DBSCAN(eps=threshold, min_samples=min_samples)
    clusters = db.fit(frames[['x', 'y']])
    
    frames.loc[:, 'labels'] = clusters.labels_
    
    for index, row in frames.iterrows():
        
        if row['labels'] == -1:
            max_id += 1
            row['labels'] = max_id
            
        
    
    
        

    
    data[data['frame_number'].isin(frame_number)].loc[:, 'labels'] = clusters.labels_



        

df = pd.DataFrame(last_positions)

/tmp/ipykernel_9564/147489572.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frames['labels'] = clusters.labels_
/tmp/ipykernel_9564/147489572.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frames['labels'] = clusters.labels_
/tmp/ipykernel_9564/147489572.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

      frame_number   id          class      conf            x            y  \
0                1    1  small vehicle  0.802636  3148.414062  1927.137207   
1                1    2  small vehicle  0.795741  3054.048584   463.554932   
2                1    3  large vehicle  0.775441  3079.295166  1646.043091   
3                1    4  small vehicle  0.775234  3848.507080   297.390717   
4                1    5  small vehicle  0.774091   238.860870  1675.974121   
...            ...  ...            ...       ...          ...          ...   
4991            20  236  small vehicle  0.010482  2584.306396  1840.970581   
4992            20  237  small vehicle  0.010436  3528.392090   323.727386   
4993            20  238  small vehicle  0.010325  2877.167969   732.392273   
4994            20  239  small vehicle  0.010027  3159.583008  1007.554504   
4995            20  240  small vehicle  0.010024  1027.523193   930.802307   

               w          h         r  labels  
0      64.58574

/tmp/ipykernel_9564/147489572.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frames['labels'] = clusters.labels_
/tmp/ipykernel_9564/147489572.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frames['labels'] = clusters.labels_
/tmp/ipykernel_9564/147489572.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

      frame_number   id          class      conf            x            y  \
768              4    1  small vehicle  0.801999  3148.131348  1926.890381   
769              4    2  small vehicle  0.791450  3915.427979   233.734512   
770              4    3  small vehicle  0.791275  3054.062988   463.645020   
771              4    4  small vehicle  0.777145  3848.352539   297.411499   
772              4    5  small vehicle  0.766359  3884.048340  1734.164551   
...            ...  ...            ...       ...          ...          ...   
5730            23  248  small vehicle  0.010434  3819.035156   575.346375   
5731            23  249  small vehicle  0.010325  2700.703857   567.679077   
5732            23  250  small vehicle  0.010264   398.754211   730.722229   
5733            23  251  small vehicle  0.010198  1169.902954  2129.196533   
5734            23  252  small vehicle  0.010189  2714.968750   565.602539   

              w          h         r  labels  
768   64.680557 

/tmp/ipykernel_9564/147489572.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frames['labels'] = clusters.labels_
/tmp/ipykernel_9564/147489572.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frames['labels'] = clusters.labels_
/tmp/ipykernel_9564/147489572.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

...

       frame_number   id          class      conf            x            y  \
15811            63    1  small vehicle  0.816391   235.939133  1672.701904   
15812            63    2  small vehicle  0.767609  3050.098877   462.156128   
15813            63    3  small vehicle  0.765997  3719.920654  1602.979370   
15814            63    4  small vehicle  0.765606  3805.991211  1739.118408   
15815            63    5  small vehicle  0.752176  3144.466064  1926.978394   
...             ...  ...            ...       ...          ...          ...   
21215            82  268  small vehicle  0.010366  3333.724121  2103.864258   
21216            82  269  small vehicle  0.010302   293.314514   794.878967   
21217            82  270  large vehicle  0.010110  2919.206299  1820.226685   
21218            82  271  small vehicle  0.010060  3422.879150   675.263367   
21219            82  272  small vehicle  0.010001   663.047119   818.368713   

               w          h         r  labels  
158

/tmp/ipykernel_9564/147489572.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frames['labels'] = clusters.labels_
/tmp/ipykernel_9564/147489572.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frames['labels'] = clusters.labels_


       frame_number   id          class      conf            x            y  \
16306            65    1  small vehicle  0.819521   236.466827  1672.962402   
16307            65    2  small vehicle  0.771110  3050.115723   462.100494   
16308            65    3  small vehicle  0.762213  3804.971924  1738.204346   
16309            65    4  small vehicle  0.761446  3363.150391  2044.668213   
16310            65    5  small vehicle  0.758357  3720.018799  1602.910522   
...             ...  ...            ...       ...          ...          ...   
21757            84  262  small vehicle  0.010444  3954.744385  1737.361938   
21758            84  263  small vehicle  0.010400  1322.877808  1947.261963   
21759            84  264  small vehicle  0.010342  2825.404297   545.522217   
21760            84  265  small vehicle  0.010299  3422.218262   675.056458   
21761            84  266  small vehicle  0.010247   331.400909   430.714050   

               w          h         r  labels  
163

/tmp/ipykernel_9564/147489572.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frames['labels'] = clusters.labels_


KeyboardInterrupt: 

In [2]:

class_colors = {
    'small vehicle': (0, 255, 255),
    'large vehicle': (255, 255, 0),
    # Ajouter d'autres classes et couleurs au besoin
}

threshold = 10

look_back = 20

max_id = 1

cap = cv2.VideoCapture(video_path)

last_positions = []

# Lire la vidéo frame par frame
for frame_number in range(data['frame_number'].min(), data['frame_number'].max() + 1):

    # Filtrer les détections pour le frame actuel
    detections_in_frame = data[data['frame_number'] == frame_number]

    if len(last_positions) < look_back :
        
        vect = np.concatenate([np.concatenate([p.get('x'), p.get('y')]) for p in last_positions])
            
        last_positions.append({'center': detection_center,'frame_number': frame_number, 'id': id, 'class': class_name, 'conf': confidence, 'x': cx, 'y': cy, 'w': w, 'h': h, 'r': r})
        

df = pd.DataFrame(last_positions)

KeyboardInterrupt: 